5. Pessoas que usam crypto fazem menos churn


In [ ]:
from google.colab import auth
import pandas as pd

# get credentials
auth.authenticate_user()
query = '''
WITH users_churn AS(
SELECT
user_id,
DATE_DIFF(TIMESTAMP('2019-05-16'),MAX(TIMESTAMP(transactions.created_date)), DAY) AS days_since_last_transaction,
DATE_DIFF(TIMESTAMP('2019-05-16'),MAX(TIMESTAMP(users.created_date)), DAY) AS days_since_creation

FROM `e-tensor-411113.neobankclean.userdevice_clean` AS users
LEFT JOIN `e-tensor-411113.neobankclean.transactions_clean` AS transactions
USING (user_id)
GROUP BY user_id
),

transacoes_usd_eur AS (
SELECT distinct user_id

FROM `e-tensor-411113.neobankclean.transactions_clean`
WHERE transactions_currency IN ('USD','EUR')
)

SELECT
u.user_id,
CASE
    WHEN days_since_last_transaction >= 180 AND days_since_creation >=180 THEN 'churner'
    ELSE 'not churner'
    END AS churner,
CASE
    WHEN t.user_id IS NULL THEN 'not usd_eur user'
    ELSE 'usd_eur user'
    END AS usd_eur_user

FROM users_churn u
LEFT JOIN transacoes_usd_eur t
  ON u.user_id = t.user_id
'''

df = pd.read_gbq(query, project_id="neural-tangent-411113")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17512 entries, 0 to 17511
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   user_id       17512 non-null  object
 1   churner       17512 non-null  object
 2   usd_eur_user  17512 non-null  object
dtypes: object(3)
memory usage: 410.6+ KB


In [ ]:
pivot = pd.pivot_table(df,
                       values=['churner','usd_eur_user'],
                       index=['churner'], columns=['usd_eur_user'], aggfunc='count',
                       dropna=True)

In [ ]:
pivot

user_id             
usd_eur_user not usd_eur user usd_eur user
churner                                   
churner                  1265         1547
not churner              4644        10056

In [ ]:
# 1265/2812 ~= 44.9%
# dentre os churners, 55.1% são usd_eur users

# 4644/14700 ~= 31.6%
# dentre os not churners, 68.4% são usd_eur users

HO: Não há relação entre usd_eur users e churners

H1: Há relação entre usd_eur users e churners

In [ ]:
import scipy.stats as stats

# Teste Chi quadrado
chi_res = stats.chi2_contingency(pivot)

print(f'p-value: {chi_res[1]}')

p-value: 5.78087400428661e-43




*   0 <= p-value <= 1
*   Se p-value < valor de significância: H0 é rejeitada
*   Fixando o valor de significância em 0.05 (valor comumente usado na literatura), temos:
*   p-value < 0.05: H0 rejeitada ⇒ H1 aceita


